In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
import sys
sys.path.append("C:/Users/clementine.rosier/OneDrive - Ekimetrics/Documents/GitHub/dynamic_pricing")

%load_ext autoreload
%autoreload 2

In [3]:
from src.simulations import SimpleSimulation
from src.simulation_melange_gaussien import GSumSimulation
from src.simulation_melange_gaussien import GMixSimulation
from src.simulation_uniforme import UniformeSimulation
from src.simple_bandit import SimpleBandit
from src.binomial_bandit import BinomialBandit
from src.simulations_contexte import ContextSimulation


ModuleNotFoundError: No module named 'src.simulations_contexte'

In [ ]:
simulation = SimpleSimulation(mu = 60, sigma=10)
simulation.optimal_price

In [ ]:
sim_u = UniformeSimulation(35,60)
sim_u.optimal_price

In [ ]:
sim_sum=GSumSimulation(45,12,20,2)
sim_sum.optimal_price

In [ ]:
sim_mix=GMixSimulation(45,12,80,2,0.25)
sim_mix.optimal_price

In [ ]:
beta_c=np.array([5,2,3])
mu_c=np.array([3,50,87])
sigma_c=np.array([1,9,10])
beta_d=np.array([[1,-1,0],[2,100,-1]])
n=np.array([2,3])
mu_e=-300
sigma_e=15

sim_context =ContextSimulation(beta_c,mu_c,sigma_c,beta_d,n,mu_e,sigma_e)

In [ ]:
def plot_demand(simulation, min_price = 0, max_price = 150, n_points = 50, n_estimates_per_point = 40) : 
    prices = []
    revenues = []
    for price in np.linspace(min_price, max_price, n_points) : 
        revenues.extend( [int(simulation.evaluate(price)) * price for x in range(n_estimates_per_point)] )
        prices.extend([price for x in range(n_estimates_per_point)])
    
    a = np.array([prices, revenues])

    fig = plt.figure(figsize = (15,6), facecolor="w")
    ax = sns.lineplot(a[0], a[1])
    ax.legend("")

In [ ]:
plot_demand(simulation)
plot_demand(sim_u)
plot_demand(sim_sum)
plot_demand(sim_mix)


In [ ]:
plot_demand(sim_context,0,  400, 50,  40)


In [ ]:
for _  in range(5):
    sim_context._simulate()
    print (sim_context.context[1])

### Try simple bandit algorithm on the simulations

In [ ]:
size = 30

k_p = np.linspace(20,80,size)
alpha_0 = np.repeat(6,size)
beta_0 = np.repeat(6,size)

bandit = BinomialBandit(k_p, alpha_0, beta_0)

In [ ]:
def test_bandit(simulation):
    hist = []
    bandit = BinomialBandit(k_p, alpha_0, beta_0)
    # Exploration round
    for i in range(size * 40):
        for j in range(bandit.k):
            bandit.chose_action(force_action=j)
            price = k_p[bandit.action]
            reward = int(simulation.evaluate(price)) * price
            bandit.update(bandit.action, reward)

    for i in range(1000):
        bandit.chose_action(method="thompson")
        price = k_p[bandit.action]
        reward = int(simulation.evaluate(price)) * price
        bandit.update(bandit.action, reward)
        hist.append(bandit.action)
    return hist

In [ ]:
#simulation somme de normale
hist_sum=test_bandit(sim_sum)
occ = sorted(Counter(hist_sum).items(), key = lambda k : k[1])
for index, nb_occ in occ : 
    print(f"Bucket {index} -- Nb occ {nb_occ} -- Price {bandit.k_p[index]}")

In [ ]:
#simulation uniforme
hist_u=test_bandit(sim_u)
occ = sorted(Counter(hist_u).items(), key = lambda k : k[1])
for index, nb_occ in occ : 
    print(f"Bucket {index} -- Nb occ {nb_occ} -- Price {bandit.k_p[index]}")

In [ ]:
#simulation mélange gaussien
hist_mix=test_bandit(sim_mix)
occ = sorted(Counter(hist_mix).items(), key = lambda k : k[1])
for index, nb_occ in occ : 
    print(f"Bucket {index} -- Nb occ {nb_occ} -- Price {bandit.k_p[index]}")